# **Computer vision and convolutional neural networks in tensorflow**

## **Inspect the data**

The data are downloaded locally. They are not loaded from the internet as is in the video.

### **Checking number of directories and files**

In [2]:
!ls

README.md   Section-03	Section-05	     command.txt
Section-02  Section-04	Test_Notebook.ipynb


In [4]:
!ls Section-05/pizza_steak/

test  train


In [5]:
!ls pizza_steak/train

ls: cannot access 'pizza_steak/train': No such file or directory


In [6]:
!ls pizza_steak/train/pizza

ls: cannot access 'pizza_steak/train/pizza': No such file or directory


In [7]:
!ls pizza_steak/train/steak

ls: cannot access 'pizza_steak/train/steak': No such file or directory


In [8]:
import os

len(os.listdir('pizza_steak/train/pizza'))

FileNotFoundError: [Errno 2] No such file or directory: 'pizza_steak/train/pizza'

In [9]:
for dirpath, dirnames, filenames in os.walk('pizza_steak/'):
    print(f"There are {len(dirnames)} direcrtories and {len(filenames)} images in {dirpath}")

### **Visualising images**

#### Getting class names programmaticaly - First approach

In [12]:
import pathlib
import numpy as np

data_lib = pathlib.Path('Section-05/pizza_steak/train')
classes_1st = np.array(sorted([item.name for item in data_lib.glob("*")]))
print(classes_1st)

['pizza' 'steak']


#### Getting class names programmaticaly - Second approach

In [14]:
classes_2nd = os.listdir('Section-05/pizza_steak/train/')
print(classes_2nd)

['pizza', 'steak']


#### Coding the visualisation

In [15]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [16]:
def view_random_image(target_dir, target_class):
    target_folder = target_dir + '/' + target_class

    random_image = random.sample(os.listdir(target_folder), 1)[0]

    img = mpimg.imread(target_folder + '/' + random_image)
    plt.imshow(img)
    plt.title(f'{target_class} ({random_image})')
    plt.axis('off')

    return img

In [17]:
img = view_random_image('pizza_steak/train', 'pizza')
print(img.shape)

FileNotFoundError: [Errno 2] No such file or directory: 'pizza_steak/train/pizza'

#### Converting an image to a tensor

In [18]:
import tensorflow as tf

In [19]:
# img_tensor = tf.constant(img)
# img_tensor

In [20]:
# Normalizing the tensor (Get all the pixel values between 0 & 1)
# img_tensor/255

## **Creating the initial convolutional network model**

### Prepare the data

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [22]:
tf.random.set_seed(42)

In [23]:
train_datagen = ImageDataGenerator(rescale = 1/255)
test_datagen = ImageDataGenerator(rescale = 1/255)

In [25]:
train_data_dir = 'Section-05/pizza_steak/train'
test_data_dir = 'Section-05/pizza_steak/test'

In [26]:
train_data = train_datagen.flow_from_directory(
    directory = train_data_dir,
    batch_size = 8,
    target_size = (224, 224),
    class_mode = 'binary',
    seed = 42,
    shuffle=False
)

Found 1500 images belonging to 2 classes.


In [27]:
test_data = test_datagen.flow_from_directory(
    directory = test_data_dir,
    batch_size = 4,
    target_size = (224, 224),
    class_mode = 'binary',
    seed = 42,
    shuffle=False
)

Found 500 images belonging to 2 classes.


### Define the model

In [28]:
model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(
        filters = 10,
        kernel_size = 3,
        activation = 'relu',
        input_shape = (224, 224, 3)
    ),
    tf.keras.layers.Conv2D(
        filters = 10,
        kernel_size = 3,
        activation = 'relu'
    ),
    tf.keras.layers.MaxPool2D(
        pool_size = 2,
        padding = 'valid'
    ),
    # tf.keras.layers.Conv2D(
    #     filters = 10,
    #     kernel_size = 3,
    #     activation = 'relu'
    # ),
    # tf.keras.layers.Conv2D(
    #     filters = 10,
    #     kernel_size = 3,
    #     activation = 'relu'
    # ),
    # tf.keras.layers.MaxPool2D(
    #     pool_size = 2
    # ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

2025-05-14 09:11:12.462711: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-14 09:11:12.465216: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-05-14 09:11:12.465502: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-14 09:11:12.465533: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 12.00GiB deviceMemoryBand

In [29]:
model_1.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy']
)

### Fit the model

In [30]:
history_1 = model_1.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

2025-05-14 09:11:55.441594: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2025-05-14 09:11:55.443849: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 4699905000 Hz
2025-05-14 09:11:55.622287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Epoch 1/5


2025-05-14 09:11:56.215888: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2025-05-14 09:13:21.313741: W tensorflow/stream_executor/gpu/asm_compiler.cc:63] Running ptxas --version returned 256
2025-05-14 09:13:21.375127: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


188/188 [==============================] - 105s 78ms/step - loss: nan - accuracy: 0.5367 - val_loss: nan - val_accuracy: 0.5000
Epoch 2/5
188/188 [==============================] - 9s 49ms/step - loss: nan - accuracy: 0.5193 - val_loss: nan - val_accuracy: 0.5000
Epoch 3/5
188/188 [==============================] - 9s 49ms/step - loss: nan - accuracy: 0.5304 - val_loss: nan - val_accuracy: 0.5000
Epoch 4/5
188/188 [==============================] - 9s 48ms/step - loss: nan - accuracy: 0.5579 - val_loss: nan - val_accuracy: 0.5000
Epoch 5/5
188/188 [==============================] - 9s 48ms/step - loss: nan - accuracy: 0.4908 - val_loss: nan - val_accuracy: 0.5000


In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

2025-05-14 09:08:49.920287: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


TensorFlow version: 2.4.1
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2025-05-14 09:08:52.162067: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-05-14 09:08:52.167439: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2025-05-14 09:08:52.597098: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:927] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-05-14 09:08:52.597153: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3060 computeCapability: 8.6
coreClock: 1.837GHz coreCount: 28 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 335.32GiB/s
2025-05-14 09:08:52.597177: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2025-05-14 09:08:52.631466: I tensorflow/stream_executor/platf